In [ ]:
# imports

# external modules
import sys
import os
import json
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib

# local modules
import omstools
importlib.reload(omstools)

In [ ]:
# load occupancy

mes = ({
    'PXDisk_+1': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+1',
    'PXDisk_+2': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+2',
    'PXDisk_+3': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_+3',
    'PXDisk_-1': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-1',
    'PXDisk_-2': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-2',
    'PXDisk_-3': 'PixelPhase1-Tracks-PXForward-clusterposition_xy_ontrack_PXDisk_-3'
})
eras = ([
    'Run2023C-PromptReco-v1',
    'Run2023C-PromptReco-v2',
    'Run2023C-PromptReco-v3',
    'Run2023C-PromptReco-v4',
    'Run2023D-PromptReco-v1',
    'Run2023D-PromptReco-v2',
    'Run2023E-PromptReco-v1',
    'Run2023F-PromptReco-v1',
])

info = {}
for era in eras:
    info[era] = {}
    for melabel, mename in mes.items():
        fname = '../data/data/ZeroBias-{}-DQMIO-{}.parquet'.format(era, mename)
        df = pd.read_parquet(fname, columns=['fromrun', 'fromlumi', 'entries'])
        runs = np.array(df['fromrun'])
        lumis = np.array(df['fromlumi'])
        entries = np.array(df['entries'])
        info[era][melabel] = {'runs': runs, 'lumis': lumis, 'entries': entries}

In [ ]:
# load oms json

omsjsons = {}
for era in eras:
    omsfile = 'omsdata/omsdata_{}.json'.format(era.replace('-PromptReco-','-'))
    if '2023E' in era: omsfile = omsfile.replace('-v1', '')
    if '2023F' in era: omsfile = omsfile.replace('-v1', '')
    with open(omsfile, 'r') as f:
        omsjsons[era] = json.load(f)

In [ ]:
# define help functions for plotting

def occupancy_scatter_plot(info, omsattr, mes, eras, 
                           xaxlabel='auto', yaxlabel='Occupancy',
                           ymax=None, normalize=False):
    # define colors
    cmap = mpl.colormaps['viridis']
    colors = [cmap(i) for i in np.linspace(0., 1., num=len(eras), endpoint=True)]
    if len(eras)==1: colors = ['b']
    # initialize figure
    fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(18,6), squeeze=False)
    axs = axs.flatten()
    for meidx, melabel in enumerate(mes.keys()):
        ax = axs[meidx]
        for eraidx, era in enumerate(eras):
            runs = info[era][melabel]['runs']
            lumis = info[era][melabel]['lumis']
            entries = info[era][melabel]['entries']
            values = omstools.find_oms_attr_for_lumisections(runs, lumis, omsjsons[era], omsattr)
            if normalize: entries = np.divide(entries, np.where(values>0, values, 1))
            ax.scatter(values, entries, s=1, alpha=0.25, color=colors[eraidx])
            if len(eras)==1: ax.text(0.05, 0.95, era + ', ' + melabel, va='top', transform=ax.transAxes, fontsize=12)
        if xaxlabel=='auto': xaxlabel = omsattr
        ax.set_xlabel(xaxlabel, fontsize=15)
        ax.set_ylabel(yaxlabel, fontsize=15)
        if ymax is not None: ax.set_ylim((0, ymax))
        fig.subplots_adjust(bottom=-0.2, left=-0.2)
    return fig, axs

In [ ]:
# make plots for pileup

# per era
for era in eras:
    fig, axs = occupancy_scatter_plot(info, 'pileup', mes, [era], xaxlabel='Pileup', yaxlabel='Occupancy')
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'pileup', mes, eras, xaxlabel='Pileup', yaxlabel='Occupancy', ymax=2e5)
for idx,ax in enumerate(axs): ax.text(0.05, 0.95, 'Run 2023, ' + list(mes.keys())[idx], va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plot for pileup with normalized occupancy

fig, axs = occupancy_scatter_plot(info, 'pileup', mes, eras, xaxlabel='Pileup', yaxlabel='Occupancy', normalize=True, ymax=5e3)
for idx,ax in enumerate(axs): ax.text(0.05, 0.95, 'Run 2023, ' + list(mes.keys())[idx], va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# investigate lumisections with zero pileup and nonzero occupancy

era = 'Run2023F-PromptReco-v1'
runs = info[era][melabel]['runs']
lumis = info[era][melabel]['lumis']
entries = info[era][melabel]['entries']
pileup = omstools.find_oms_attr_for_lumisections(runs, lumis, omsjsons[era], 'pileup')
inds = np.nonzero((pileup==0) & (entries!=0))[0]
print(len(inds))
print(set(runs[inds]))
for run, lumi in zip(runs[inds], lumis[inds]): print('{} {}'.format(run, lumi))

# for 2023D-v1: 31 lumisections, runs 370497, 370169, 370172, 370357
#               OMS: - isolated semi-random lumisections with "0 pileup", probably OMS error,
#                      can take average of previous and next lumisection? or just filter out these lumisections?
#                    - other cases unclear, delivered lumi and pileup are 0, but still nonzero occupancy,
#                      maybe DAQ or trigger issues?
# other eras: only small number of lumisections, seem to be mostly isolated semi-random OMS errors.
# note: era 2023E has no instances zero pileup and nonzero occupancy, just very low pileup (O(0.1))

In [ ]:
# make plots for delivered lumi

# per era
for era in eras:
    fig, axs = occupancy_scatter_plot(info, 'delivered_lumi_per_lumisection', mes, [era], 
                                      xaxlabel='Delivered luminosity per lumisection (/pb)', 
                                      yaxlabel='Occupancy')
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'delivered_lumi_per_lumisection', mes, eras,
                                  xaxlabel='Delivered luminosity per lumisection (\pb)', 
                                  yaxlabel='Occupancy', ymax=2e5)
for idx,ax in enumerate(axs): ax.text(0.05, 0.95, 'Run 2023, ' + list(mes.keys())[idx], va='top', transform=ax.transAxes, fontsize=12)

In [ ]:
# make plots for recorded lumi

# per era
for era in eras:
    fig, axs = occupancy_scatter_plot(info, 'recorded_lumi_per_lumisection', mes, [era], 
                                      xaxlabel='Recorded luminosity per lumisection (/pb)', 
                                      yaxlabel='Occupancy')
    
# eras together
fig, axs = occupancy_scatter_plot(info, 'recorded_lumi_per_lumisection', mes, eras,
                                  xaxlabel='Recorded luminosity per lumisection (\pb)', 
                                  yaxlabel='Occupancy', ymax=2e5)
for idx,ax in enumerate(axs): ax.text(0.05, 0.95, 'Run 2023, ' + list(mes.keys())[idx], va='top', transform=ax.transAxes, fontsize=12)